# Small Concept Model (SCM) Training
Here, we train the small concept model for next-concept prediction.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from datasets import load_dataset
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

load_cached_embeddings = "saved_models/train_seq_embeddings.pt"

/anaconda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Dataloader

In [2]:
data = load_dataset("francescoortame/bookcorpus-sorted-100k16x", split="train")

Create the embeddings to cache them for training.

In [2]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

if load_cached_embeddings is None:
    flat_texts = [item for sublist in data["slice"] for item in sublist]
    embeddings = encoder.encode(flat_texts, batch_size=32, show_progress_bar=True, convert_to_tensor=True, device=device)

Reshape the embeddings to preserve the original sequence structure and create the training dataloader.

In [3]:
batch_size = 16

if load_cached_embeddings is None:
    reshaped_embeddings = embeddings.contiguous().view(100000, 16, 384)

else:
    reshaped_embeddings = torch.load(load_cached_embeddings)

train_ds = TensorDataset(reshaped_embeddings[:int(0.9*len(reshaped_embeddings))])
valid_ds = TensorDataset(reshaped_embeddings[int(0.9*len(reshaped_embeddings)):])

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_ds, batch_size=batch_size, shuffle=True)

Finally, define a function to compute the causal mask for autoregressive modeling.

In [4]:
def generate_causal_mask(seq_len: int, device: torch.device):
    mask = torch.triu(torch.ones(seq_len, seq_len, device=device), diagonal=1)
    mask = mask.masked_fill(mask == 1, float("-inf"))
    mask = mask.masked_fill(mask == 0, float(0.0))
    return mask  # [seq_len, seq_len]

---

## Model

In [5]:
class PreNet(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int):
        super(PreNet, self).__init__()
        self.linear = nn.Linear(input_dim, hidden_dim)
        self.scaler_mean = 0.0
        self.scaler_std = 1.0

    def normalize(self, x):
        return (x - self.scaler_mean) / self.scaler_std

    def forward(self, x):
        x = self.normalize(x)
        return self.linear(x)


class PostNet(nn.Module):
    def __init__(self, hidden_dim: int, output_dim: int):
        super(PostNet, self).__init__()
        self.linear = nn.Linear(hidden_dim, output_dim)
        self.scaler_mean = 0.0
        self.scaler_std = 1.0

    def denormalize(self, x):
        return x * self.scaler_std + self.scaler_mean

    def forward(self, x):
        x = self.linear(x)
        return self.denormalize(x)


class TransformerDecoder(nn.Module):
    def __init__(self, hidden_dim: int, num_heads: int, num_layers: int, ff_dim: int, dropout: float = 0.1, max_seq_len: int = 16):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.TransformerDecoderLayer(
                d_model=hidden_dim,
                nhead=num_heads,
                dim_feedforward=ff_dim,
                dropout=dropout
            )
            for _ in range(num_layers)
        ])
        self.pos_encoder = nn.Parameter(torch.zeros(1, max_seq_len, hidden_dim))

    def forward(self, x, tgt_mask=None):
        """
        x: Tensor of shape [B, T, hidden_dim]
        tgt_mask: square mask of shape [T, T] containing 0 for allowed, -inf for masked.
        """
        seq_len = x.size(1)
        # Add positional encoding (broadcasted over batch dimension)
        x = x + self.pos_encoder[:, :seq_len, :]
        # TransformerDecoderLayer in PyTorch expects input shape [T, B, hidden_dim], so we must transpose.
        # Indeed, nn.TransformerDecoderLayer expects (tgt, memory, ...), each of shape [T, B, E].
        # We’re using “decoder‐only” (no external memory), so we feed the same x as both tgt and memory.
        # That forces it to attend “only to past” via tgt_mask.
        x = x.transpose(0, 1)  # now shape [T, B, hidden_dim]
        for layer in self.layers:
            x = layer(tgt=x, memory=x, tgt_mask=tgt_mask)
        x = x.transpose(0, 1)  # back to [B, T, hidden_dim]
        return x


class SimpleLCM(nn.Module):
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int,
        num_heads: int,
        num_layers: int,
        ff_dim: int,
        output_dim: int,
        dropout: float = 0.1
    ):
        super(SimpleLCM, self).__init__()
        self.prenet = PreNet(input_dim, hidden_dim)
        self.transformer = TransformerDecoder(hidden_dim, num_heads, num_layers, ff_dim, dropout)
        self.postnet = PostNet(hidden_dim, output_dim)

    def forward(self, x):
        # x: [B, 16, input_dim]
        x = self.prenet(x)            # [B, 16, hidden_dim]
        causal_mask = generate_causal_mask(x.size(1), device=x.device)  # [16, 16]
        x = self.transformer(x, tgt_mask=causal_mask)  # [B, 16, hidden_dim]
        x = self.postnet(x)           # [B, 16, output_dim]
        return x

Initialize the model.

In [ ]:
model = SimpleLCM(
    input_dim=384,
    hidden_dim=512,
    num_heads=4,
    num_layers=4,
    ff_dim=4*512,
    output_dim=384,
    dropout=.2
)
model.to(device)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Trainable parameters: {trainable_params}")

Trainable parameters: 38410368


## Training

In [10]:
num_epochs =  10
lr = 1e-4

optimizer = optim.Adam(model.parameters(), lr=lr)

In [11]:
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}")
    model.train()
    total_loss = 0.0

    for (batch_input,) in tqdm(train_loader, total=len(train_loader)):
        # batch_input: [B, 16, input_dim]
        batch_input = batch_input.to(device)  # move to GPU if available
        
        # (No separate tgt_in/tgt_out since we’re teacher-forcing with the same x for everything.)
        optimizer.zero_grad()
        
        # 1) PreNet → TransformerDecoder → PostNet
        # The transformer’s forward handles the causal mask internally.
        preds = model(batch_input)  # [B, 16, output_dim]
        
        # 2) Build a mask to zero out loss at t=0
        # We do not care about predicting position 0 (no “previous” vector), so ignore it:
        mask = torch.ones_like(batch_input)  # shape [B, 16, input_dim]
        mask[:, 0, :] = 0.0  # zero‐weight the t=0 position

        # 3) Compute MSE only on positions 1..15
        loss = F.mse_loss(preds * mask, batch_input * mask, reduction="sum")
        # If you want mean‐per‐element: 
        #   num_pred_steps = batch_input.size(0) * (batch_input.size(1) - 1)  # B * 15
        #   loss = loss / num_pred_steps

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader.dataset)  # if you used reduction="sum"

    with torch.no_grad():
        model.eval()
        total_val_loss = 0.0

        for (batch_input, ) in tqdm(valid_loader, total=len(valid_loader)):
            batch_input = batch_input.to(device)
            preds = model(batch_input)
            mask = torch.ones_like(batch_input)
            mask[:, 0, :] = 0.0

            loss = F.mse_loss(preds * mask, batch_input * mask, reduction="sum")
            total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(valid_loader)

    print(f"Epoch {epoch+1:03d} | Train Loss: {avg_loss:.5f} | Valid Loss: {avg_val_loss:.5f} ")

Epoch 1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 625/625 [00:04<00:00, 148.53it/s]


Epoch 001 | Train Loss: 12.38503 | Valid Loss: 7.35501 
Epoch 2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 625/625 [00:04<00:00, 147.72it/s]


Epoch 002 | Train Loss: 0.31728 | Valid Loss: 0.28061 
Epoch 3


...

Epoch 008 | Train Loss: 0.00386 | Valid Loss: 0.04614 
Epoch 9


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 625/625 [00:04<00:00, 147.95it/s]


Epoch 009 | Train Loss: 0.00257 | Valid Loss: 0.00954 
Epoch 10


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 625/625 [00:04<00:00, 148.25it/s]

Epoch 010 | Train Loss: 0.00212 | Valid Loss: 0.00575 


## Inference

### Functions and Preparations for Inference

In [12]:
from modules.prenet import PreNet
from modules.encdec import get_gpt2_decoder

decoder, tokenizer = get_gpt2_decoder()

prenet = PreNet(
    input_dim=384,
    output_dim=768,
    bottleneck_dim=128,
    prefix_len=20
).to(device)

prenet.load_state_dict(torch.load("saved_models/prenet_prefix_tuning_bookcorpus.pth", map_location=device))

def generative_inference(model, initial_sequence, n_future_steps):
    """
    Args:
        model: Trained SimpleLCM model.
        initial_sequence: Tensor of shape (k, input_dim)
        n_future_steps: How many future steps to generate
    Returns:
        Tensor of shape (k + n_future_steps, input_dim)
    """
    model.eval()

    input_seq = initial_sequence.clone().unsqueeze(0).to(device)  # (1, k, input_dim)
    generated = []

    with torch.no_grad():
        for _ in range(n_future_steps):
            output = model(input_seq)           # (1, seq_len, output_dim)
            next_pred = output[:, -1, :]        # (1, output_dim)
            generated.append(next_pred.squeeze(0))  # (output_dim,)
            input_seq = torch.cat([input_seq, next_pred.unsqueeze(1)], dim=1)

    generated = torch.stack(generated, dim=0)  # (n_future_steps, output_dim)
    full_sequence = torch.cat([initial_sequence.to(device), generated], dim=0)  # (k + n_future_steps, input_dim)
    return full_sequence

def vec_to_text(embedding, decoder, tokenizer, prenet, gen_len=50):
    """
    Given input text, encode it, generate prefix via PreNet, and autoregressively decode output text.
    """
    decoder.eval()
    prenet.eval()
    with torch.no_grad():
        prefix = prenet(embedding.unsqueeze(0))  # (1, prefix_len, model_dim)

        generated = prefix  # initial embeddings
        generated_ids = []
        for _ in range(gen_len):
            outputs = decoder(inputs_embeds=generated)
            next_logits = outputs.logits[:, -1, :]
            next_id = torch.argmax(next_logits, dim=-1).unsqueeze(-1)  # greedy
            generated_ids.append(next_id)
            next_embed = decoder.transformer.wte(next_id)
            generated = torch.cat([generated, next_embed], dim=1)

    gen_ids = torch.cat(generated_ids, dim=1)
    return tokenizer.decode(gen_ids[0].cpu().numpy(), skip_special_tokens=True)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


### Generation Inference

In [1]:
future_steps = 3

sentences = [
    "hello, my name is jack.",
    "how are you?"
]
encoded_sentences = encoder.encode(sentences, convert_to_tensor=True)
generated_seq = generative_inference(model, encoded_sentences, n_future_steps=future_steps)

for vec in generated_seq:
    generated_text = vec_to_text(vec, decoder, tokenizer, prenet, 50)
    print(generated_text)

NameError: name 'encoder' is not defined